# Observational Selection Functions

**Author:** Alex Drlica-Wagner ([@kadrlica](https://github.com/des-science/mw-sats/issues/new?body=@kadrlica))<br>
**Date:** 2019-11-24<br>
**Tag:** v0.2<br>

This notebook is intended to demonstrate the application of the machine learning model of the observational selection function for Milky Way satellites derived in [Drlica-Wagner, Bechtol et al. (2019)](...). The selection function is distributed as a [XGBoost](https://xgboost.readthedocs.io/en/latest/) model file. While this example is written in Python, the model should be accessible with any language supported by XGBoost (i.e., R, C++, Julia, etc.).

## Installation

As a simple demonstration, we try to keep the installation to a minumum. To get started, clone this GitHub repository and change into the newly created directory:

```
> git clone https://github.com/des-science/mw-sats.git
> cd mw-sats
```

We provide a minimal conda environment (`conda-env.yaml`) that will let you execute this tutorial. You can create this environment with:
```
> conda env create -v -f conda-env.yaml
```
Once the environment has been installed, you can activate it and start this notebook server on your local machine.
```
> conda activate mw-sats
> jupyter notebook ExampleNotebook.ipynb
```

Next, we download the model files and other ancillary data products. These are stored on GitHub as a tarball under the [Releases](https://github.com/des-science/mw-sats/releases) page.

In [1]:
!./download.py config.yaml

wget https://github.com/des-science/mw-sats/releases/download/v0.2/mw-sats-data.tar.gz
tar -xzf mw-sats-data.tar.gz


## Setup

First we start with some generic imports.

In [2]:
from os.path import join,exists
from collections import OrderedDict as odict

import yaml
import numpy as np
import healpy as hp

Then we load a configuration file that defines file paths and global values.

In [3]:
config = yaml.safe_load(open('config.yaml'))

BITS = config['maskbits']
BADBITS = (BITS['EBV'] | BITS['ASSOC'] | BITS['STAR'] | BITS['FOOT'] | \
           BITS['FAIL'] | BITS['ART'])

def get_filepath(survey,filetype):
    """Get the filepath for a specific survey and filetype.
    
    Parameters
    ----------
    survey   : survey of interest ['des','ps1']
    filetype : type of data file
    
    Returns
    -------
    filepath : path to file
    """
    filepath = join(config['datadir'],config[survey][filetype])
    return filepath

## Minimal Example

Here we demonstrate a minimal example to load and evaluate the classifiers. 

In [4]:
import xgboost as xgb

ps1_model = xgb.XGBClassifier()
filename = get_filepath('ps1','model')
print("Loading model from %s..."%filename)
ps1_model.load_model(filename)

des_model = xgb.XGBClassifier()
filename = get_filepath('des','model')
print("Loading model from %s..."%filename)
des_model.load_model(filename)

Loading model from ./data/classifier_stellar_ps1_v13.0.1.model...
Loading model from ./data/classifier_stellar_des_v12.2.0.model...


The parameters of the classifier are: log(distance [kpc]), absolute magnitude, log(half-light radius [kpc]), and stellar density. These can be directly fed to the classifier to return a detection probability for a given survey.

In [5]:
distance        = 100.0 # Heliocentric distance [kpc]
abs_mag         = -5.0  # Absolute V-band magnitude
r_physical      = 0.1   # Physical azimuthally averaged half-light radius [kpc]
stellar_density = 1.0   # Stellar density (r < 22) [stars/arcmin^2]

x_eval = np.vstack([np.log10(distance), abs_mag, np.log10(r_physical), stellar_density]).T
print("Input parameters:", x_eval)

ps1_pdet = ps1_model.predict_proba(x_eval)[:,1]
print("PS1 detection probability:",ps1_pdet)

des_pdet = des_model.predict_proba(x_eval)[:,1]
print("DES detection probability:",des_pdet)

Input parameters: [[ 2. -5. -1.  1.]]
PS1 detection probability: [0.5263512]
DES detection probability: [0.9909887]


Input parameters can also be provided in vectorized format.

In [6]:
size            = 5                       # Number of samples
distance        = np.logspace(1,2.5,size) # Heliocentric distance [kpc]
abs_mag         = -5.0*np.ones(size)      # Absolute V-band magnitude
r_physical      = 0.1*np.ones(size)       # Physical azimuthally averaged half-light radius [kpc]
stellar_density = 1.0*np.ones(size)       # Stellar density (r < 22) [stars/arcmin^2]

x_eval = np.vstack([np.log10(distance), abs_mag, np.log10(r_physical), stellar_density]).T
print("Input parameters:\n", x_eval)

ps1_pdet = ps1_model.predict_proba(x_eval)[:,1]
print("PS1 detection probability:",ps1_pdet)

des_pdet = des_model.predict_proba(x_eval)[:,1]
print("DES detection probability:",des_pdet)

Input parameters:
 [[ 1.    -5.    -1.     1.   ]
 [ 1.375 -5.    -1.     1.   ]
 [ 1.75  -5.    -1.     1.   ]
 [ 2.125 -5.    -1.     1.   ]
 [ 2.5   -5.    -1.     1.   ]]
PS1 detection probability: [0.90075934 0.986316   0.86103725 0.36552337 0.00984656]
DES detection probability: [0.8823705  0.9935754  0.9943646  0.98358077 0.7224905 ]


## Deeper Example

The example above did not include any explicit spatial dependence (i.e., the survey footprint or stellar density at any given location on the sky). To incorporate this information, it is helpful to write a short class to encapsulate this information. This is especially useful because we don't want to have to load the large nside=4096 HEALPix mask file every time we want to evaluate the detection probability of a satellite.

In [7]:
class SurveySelectionFunction(object):
    """Survey selection function including spatial information."""

    def __init__(self, survey):
        """Initialize survey selection function."""
        self.survey = survey
        print("Loading model...")
        self.model = xgb.XGBClassifier()
        self.model.load_model(get_filepath(self.survey,'model'))
        print("Loading mask...") # This can take a while...
        self.mask = hp.read_map(get_filepath(self.survey,'mask'),dtype=int)
        print("Loading stellar density map...")
        self.density = hp.read_map(get_filepath(self.survey,'density'))

    @staticmethod
    def get_hpxmap_value(hpxmap,lon,lat):
        """Get the value of a healpix map at a given location.
        
        Parameters
        ----------
        hpxmap : HEALPix map (must be in same frame as `lon` and `lat`)
        lon : longitude [deg]
        lat : latitude [deg]
        
        Returns
        -------
        value : healpix map value
        """
        nside = hp.get_nside(hpxmap)
        pix   = hp.ang2pix(nside,lon,lat,lonlat=True)
        return hpxmap[pix]
        
    def get_stellar_density(self, ra, dec):
        """Calculate the stellar density at the location of each satellite.

        Parameters
        ----------
        ra : right ascension [deg]
        dec: declination [deg]
        
        Returns
        -------
        stellar_density : stellar density [stars/arcmin^2]
        """
        return self.get_hpxmap_value(self.density,ra,dec)

    def get_mask_value(self, ra, dec):
        """Calculate the mask value at the location of each satellite.

        Parameters
        ----------
        ra : right ascension [deg]
        dec: declination [deg]
        
        Returns
        -------
        value : mask value
        """
        return self.get_hpxmap_value(self.mask,ra,dec)
    
    def predict(self, distance, abs_mag, r_physical, ra, dec):
        """Predict the satellite detection probability for a survey.
        
        Parameters
        ----------
        distance   : Heliocentric distance [kpc]
        abs_mag    : Absolute V-band magnitude
        r_physical : Physical azimuthally averaged half-light radius [kpc]
        ra         : Right ascension [deg]
        dec        : Declination [deg]

        Returns
        -------
        pdet : detection probability
        """
        # Calculate the stellar density at the location of each satellite.
        stellar_density = self.get_stellar_density(ra,dec)
        
        # Evaluate the model for each satellite
        x_eval = np.vstack([np.log10(distance), abs_mag, np.log10(r_physical), stellar_density]).T
        pred = self.model.predict_proba(x_eval)[:,1]

        #Calculate the mask value at the location of each satellite.
        mask_value = self.get_mask_value(ra,dec)
        # Set pdet to zero for satellites in masked area
        bad = (mask_value & BADBITS) != 0
        pred[bad] = 0
        
        return pred

In [8]:
ps1_ssf = SurveySelectionFunction('ps1')

Loading model...
Loading mask...
NSIDE = 4096
ORDERING = NESTED in fits file
INDXSCHM = IMPLICIT
Ordering converted to RING
Loading stellar density map...
NSIDE = 128
ORDERING = RING in fits file
INDXSCHM = IMPLICIT


In [9]:
size       = 5                        # Number of satellites
distance   = 100*np.ones(size)        # Heliocentric distance [kpc]
abs_mag    = -5.0*np.ones(size)       # Absolute V-band magnitude
r_physical = 0.1*np.ones(size)        # Physical azimuthally averaged half-light radius [kpc]
ra         = np.linspace(40,80,size)  # Right ascension (deg)
dec        = -10.0*np.ones(size)      # Declination (deg)

print("PS1 stellar density [stars/arcmin^2]:",ps1_ssf.get_stellar_density(ra,dec))

ps1_pdet = ps1_ssf.predict(distance=distance,abs_mag=abs_mag,r_physical=r_physical,ra=ra,dec=dec)
print("PS1 detection probability:",ps1_pdet)

PS1 stellar density [stars/arcmin^2]: [0.94259155 1.08424509 1.16102922 1.68263185 2.96810436]
PS1 detection probability: [0.55837226 0.4944182  0.4825996  0.33240312 0.08314446]
